In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Soil Classification using Tensorflow 
For this first i am going to try with the meticulously preprocessed dataset 

In [ ]:
# importing modules 
import tensorflow as tf 
from PIL import Image
import matplotlib.pyplot as plt 
from tensorflow.keras.layers import Rescaling , Dense , MaxPooling2D ,Conv2D , Input , Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy as scc

2025-06-19 10:51:56.794694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750330317.054346      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750330317.121900      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TO check if the pixel size of the generated dataset are of consistent size 

In [ ]:

with Image.open("/kaggle/input/comprehensive-soil-classification-datasets/Orignal-Dataset/Black_Soil/1.jpg") as img :
    width , height = img.size
    print(f"Width: {width} pixels")
    print(f"Height: {height} pixels")

Width: 1000 pixels
Height: 544 pixels


As seen above the sizes are inconsistent 

Let us load the dataset into tensors 

In [ ]:
# let us load the modeles uing keras moudule image_datasets_from_directory 
dir = "/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset"
new_dir = "/kaggle/working/CyAUG-Cleaned2"
ih = iw = 224 # image height and image widht , so the image is of 224 pixels square 

train_ds = tf.keras.utils.image_dataset_from_directory(
    new_dir , 
    validation_split = 0.2 , 
    subset = 'training' , 
    seed = 123 , 
    image_size = (ih , iw) , 
    batch_size = 32 , 
    follow_links = True
)

val_test_ds = tf.keras.utils.image_dataset_from_directory(
  new_dir,
  validation_split=0.2, # Same split percentage
  subset="validation",
  seed=123, # Same seed for consistent split
  image_size=(ih, iw),
  batch_size = 32 
)

print(f"Number of training batches: {len(train_ds)}")
print(f"Number of validation/test batches: {len(val_test_ds)}")
   

NotFoundError: Could not find directory /kaggle/working/CyAUG-Cleaned2

In [ ]:
classes = train_ds.class_names
print(f"Names of the different types of soil are {classes}")

In [ ]:
thala = len(classes) # Thala for a reason # Suiiiiii
print("Thala " , thala)

In [ ]:
# Normalization layers used for autotuning such that tensorflow dynamically resizes the batches and stuff for the cache and according to the available memeory 
N_l = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (N_l(x), y))
val_test_ds = val_test_ds.map(lambda x, y: (N_l(x), y))

In [ ]:
for images, labels in train_ds.take(1):
  image_shape = images.shape[1:]
  print(f"Train batch image shape: {images.shape}, labels shape: {labels.shape}")

for images, labels in val_test_ds.take(1):
  print(f"Val/Test batch image shape: {images.shape}, labels shape: {labels.shape}")

In [ ]:
print(image_shape)

The batch has 32 filters due to the batch sieze and the shape of the image is (224 , 224) and 3 refers to the channels as the image is chromatic and has 4 color channels that are Red , Green and Blue 

In [ ]:
model = Sequential([
    Input(shape = image_shape) ,
    # First convolution block 
    Conv2D(
        filters = 32 , 
        kernel_size = (4 , 4) , 
        strides = (2, 2) , 
        activation = 'relu'
    ) , 
    MaxPooling2D(pool_size = (2,2)) , 
    # Second Convolution Block 
    Conv2D(
        filters = 64 , 
        kernel_size = (3 , 3) , 
        strides = (1, 1) , 
        activation = 'relu'
    ) , 
    MaxPooling2D(pool_size = (2,2)) , 
    # Classifier block 
    Flatten() , 
    Dense(
        128 , 
        activation = 'relu'
    ) , 
    Dense(
        64 , 
        activation = 'relu'
    ) , 
    Dense(
        thala , 
        activation = "softmax"
    )
    
])

In [ ]:
model.compile(
    optimizer = 'adam' , 
    loss = scc(from_logits = False) , 
    metrics = ['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_ds , 
    epochs = 100 , 
    validation_data = val_test_ds
)

In [ ]:
loss, accuracy = model.evaluate(val_test_ds, verbose=2)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.4f}")


In [ ]:
base_dataset_path = '/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset'

# Define the supported image extensions (case-insensitive)
supported_image_extensions = ['.jpeg', '.jpg', '.png', '.gif', '.bmp']

print(f"Scanning directory: {base_dataset_path} for unsupported file types...\n")

unsupported_files_found = False

# Walk through all directories and files in the specified path
for dirname, _, filenames in os.walk(base_dataset_path):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        # Get the file extension and convert to lowercase for case-insensitive comparison
        file_extension = os.path.splitext(filename)[1].lower()

        # Check if the file extension is NOT in our list of supported image extensions
        if file_extension not in supported_image_extensions:
            print(f"Unsupported file type found: {file_path}")
            unsupported_files_found = True

if not unsupported_files_found:
    print("No unsupported image file extensions (JPEG, PNG, GIF, BMP) found in the dataset.")
    print("However, this does not guarantee files are not corrupted or have incorrect headers.")

print("\nScan complete.")

In [ ]:
import os
import shutil

# Root directory of your dataset
dataset_dir = '/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset'

# Walk through all subdirectories
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith('.webp'):
            file_path = os.path.join(root, file)
            print(f"Removing unsupported file: {file_path}")
            os.remove(file_path)  # or use shutil.move(file_path, target_dir) to move instead


In [ ]:
src_dir = Path("/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset")
dst_dir = Path("/kaggle/working/CyAUG-Cleaned2")

# Create the cleaned directory
dst_dir.mkdir(parents=True, exist_ok=True)

# Supported extensions
supported_exts = ('.jpg', '.jpeg', '.png')

# File to skip (relative path from src_dir)
skip_file = os.path.join("Alluvial_Soil", "44.jpg")

# Copy loop
for root, dirs, files in os.walk(src_dir):
    for file in files:
        if file.lower().endswith(supported_exts):
            # Compute relative path to check against skip_file
            rel_path = os.path.relpath(os.path.join(root, file), src_dir)
            if rel_path == skip_file:
                print(f"❌ Skipping file: {rel_path}")
                continue

            # Destination
            src_file = os.path.join(root, file)
            dest_subdir = dst_dir / os.path.relpath(root, src_dir)
            dest_subdir.mkdir(parents=True, exist_ok=True)
            shutil.copy(src_file, dest_subdir / file)

In [ ]:
valid_exts = ('.jpg', '.jpeg', '.png')
dataset_dir = "/kaggle/input/comprehensive-soil-classification-datasets/CyAUG-Dataset"

image_paths = []
labels = []

class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)
    for fname in os.listdir(class_dir):
        if not fname.lower().endswith(valid_exts):
            continue
        fpath = os.path.join(class_dir, fname)

        # Step 1: Check with PIL
        try:
            with Image.open(fpath) as img:
                img.verify()
        except:
            print(f"❌ Skipping unreadable with PIL: {fpath}")
            continue

        # Step 2: Check with TensorFlow decode
        try:
            raw = tf.io.read_file(fpath)
            _ = tf.image.decode_image(raw, channels=3)
        except:
            print(f"❌ Skipping unreadable with tf.image: {fpath}")
            continue

        # Passed both checks, safe to use
        image_paths.append(fpath)
        labels.append(class_to_idx[class_name])